In [1]:
import pandas as pd
import requests
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bs
import urllib.request

import warnings
warnings.filterwarnings('ignore')

import re
from tqdm import tqdm 

In [2]:
check_df = pd.read_csv("./name_code_0206.csv",  dtype={'코드명': str}) # 코드명 앞에 0 유지
check_df

,날짜,순위,종목명,코드명
0,2023-02-06,1,삼성전자,005930
1,2023-02-06,2,카카오,035720
2,2023-02-06,3,SK하이닉스,000660
3,2023-02-06,4,타이거일렉,219130
4,2023-02-06,5,삼성바이오로직스,207940
5,2023-02-06,6,NAVER,035420
6,2023-02-06,7,서린바이오,038070
7,2023-02-06,8,마인즈랩,377480
8,2023-02-06,9,AP위성,211270
9,2023-02-06,10,KT&G,033780


In [3]:
check_df.loc[0]

날짜     2023-02-06
순위              1
종목명          삼성전자
코드명        005930
Name: 0, dtype: object

In [4]:
check_df.loc[0][2]

'삼성전자'

In [5]:
check_df.loc[0][3]

'005930'

In [6]:
def NS_users_crawler(pages):
    # User-Agent 설정 : 네이버 보안때문에 유저처럼 설정해줘야 보임 
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
    result_df = pd.DataFrame([])


    for i in tqdm(range(30)): # 1~30위
        list = check_df.loc[i]
        
        for page in range(1, pages+1):
            url = "https://finance.naver.com/item/board.naver?code=%s&page=%s" % (list[3], str(page)) # 해당 기업의 종토방
            # html → parsing
            html = requests.get(url, headers=headers).content
            # 한글 깨짐 방지 decode
            soup = bs(html.decode('euc-kr', 'replace'), 'html.parser')
            table = soup.find('table', {'class': 'type2'}) # 종목토론실 댓글 
            tb = table.select('tbody > tr') # 종목토론실 댓글 세부
            page = page
           
        

            for i in range(len(tb)):
                if len(tb[i].select('td > span')) > 0: # 글의 존재 여부
                    date = tb[i].select('td > span')[0].text[:10] # 글의 날짜
                    title = tb[i].select('td.title > a')[0]['title'] # 글의 제목

                    content_url = "https://finance.naver.com" + tb[i].select('td.title > a')[0]['href'] # 글 내용 읽기 위해 특정 게시물에 접속
                    content_html = requests.get(content_url, headers=headers).content
                    content_soup = bs(content_html.decode('euc-kr', 'replace'), 'html.parser')
                    content = content_soup.find('div', {'id': 'body'}).text # 글의 내용
                    
                    view = content_soup.find('span', {'class':'tah p11'}).text
                    agree = content_soup.find('strong',{'class':'tah p11 red01 _goodCnt'}).text

                    table = pd.DataFrame({'날짜': [date], '기업명':[list[2]], '제목+내용': [title + '  ' + content], '조회수' : [int(view)], '공감수' : [int(agree)], '페이지' : [page]})
                    result_df = result_df.append(table)

                    # result_df.reset_index(inplace=True)
                    # result_df.drop(["index"],axis=1, inplace=True)

    result_df["날짜"] = pd.to_datetime(result_df['날짜'], format = "%Y.%m.%d")
    # result_df.set_index("날짜", inplace=True)

    return result_df


In [7]:
data = NS_users_crawler(60) # 몇 페이지까지 볼껀가 설정 (30개 기업 1페이지에 약 2분 정도 걸림)

100%|██████████| 30/30 [1:52:21<00:00, 224.71s/it]


In [8]:
data

,날짜,기업명,제목+내용,조회수,공감수,페이지
0,2023-02-09,삼성전자,50억은 무죄야 그거 재개발 이익에서 몇푼 안되거든.\r 나눠 먹은 애들이 아직...,1,0,1
0,2023-02-09,삼성전자,누가 한민족 아니랄까봐 ㅋㅋㅋㅋㅋ 북한은 돼지딸 때매 난리고\r 남한은 개딸 때매...,5,0,1
0,2023-02-09,삼성전자,노인들 무임승차 이슈는 그냥 분노를 노인들한테 돌리는거야 뻔하지 뭐\r 야비한 수법,4,0,1
0,2023-02-09,삼성전자,2찍이들 몰살 해야해~~~~~~~~ 그래야 나라가 정상으로 간당,4,1,1
0,2023-02-09,삼성전자,빨개이들 이리와 오전 빠따마짜!!!\r일해라 일 .여기서 난동직이는기 느거일이네....,3,0,1
...,...,...,...,...,...,...
0,2022-12-28,SK아이이테크놀로지,얜 배당도 안주면서 개폭이네 ㅋ ㅋ,312,1,60
0,2022-12-28,SK아이이테크놀로지,쌓는거봐라 욕나오네,343,1,60
0,2022-12-28,SK아이이테크놀로지,"SK 배터리 탑재 아이오닉7,EV9 출시 23년은 SK의해다",478,5,60
0,2022-12-28,SK아이이테크놀로지,3만원대까지 올까요? 냉철한 분석 부탁삽니다,665,2,60


In [9]:
data = data.reset_index(drop=True)

In [10]:
data

,날짜,기업명,제목+내용,조회수,공감수,페이지
0,2023-02-09,삼성전자,50억은 무죄야 그거 재개발 이익에서 몇푼 안되거든.\r 나눠 먹은 애들이 아직...,1,0,1
1,2023-02-09,삼성전자,누가 한민족 아니랄까봐 ㅋㅋㅋㅋㅋ 북한은 돼지딸 때매 난리고\r 남한은 개딸 때매...,5,0,1
2,2023-02-09,삼성전자,노인들 무임승차 이슈는 그냥 분노를 노인들한테 돌리는거야 뻔하지 뭐\r 야비한 수법,4,0,1
3,2023-02-09,삼성전자,2찍이들 몰살 해야해~~~~~~~~ 그래야 나라가 정상으로 간당,4,1,1
4,2023-02-09,삼성전자,빨개이들 이리와 오전 빠따마짜!!!\r일해라 일 .여기서 난동직이는기 느거일이네....,3,0,1
...,...,...,...,...,...,...
35995,2022-12-28,SK아이이테크놀로지,얜 배당도 안주면서 개폭이네 ㅋ ㅋ,312,1,60
35996,2022-12-28,SK아이이테크놀로지,쌓는거봐라 욕나오네,343,1,60
35997,2022-12-28,SK아이이테크놀로지,"SK 배터리 탑재 아이오닉7,EV9 출시 23년은 SK의해다",478,5,60
35998,2022-12-28,SK아이이테크놀로지,3만원대까지 올까요? 냉철한 분석 부탁삽니다,665,2,60


In [11]:
data.to_csv('네이버_60페이지.csv')

In [7]:
data = pd.read_csv('네이버_60페이지.csv',lineterminator='\n').drop(['Unnamed: 0'], axis=1)

In [8]:
data

,날짜,기업명,제목+내용,조회수,공감수,페이지
0,2023-02-09,삼성전자,50억은 무죄야 그거 재개발 이익에서 몇푼 안되거든.\r 나눠 먹은 애들이 아직...,1,0,1
1,2023-02-09,삼성전자,누가 한민족 아니랄까봐 ㅋㅋㅋㅋㅋ 북한은 돼지딸 때매 난리고\r 남한은 개딸 때매...,5,0,1
2,2023-02-09,삼성전자,노인들 무임승차 이슈는 그냥 분노를 노인들한테 돌리는거야 뻔하지 뭐\r 야비한 수법,4,0,1
3,2023-02-09,삼성전자,2찍이들 몰살 해야해~~~~~~~~ 그래야 나라가 정상으로 간당,4,1,1
4,2023-02-09,삼성전자,빨개이들 이리와 오전 빠따마짜!!!\r일해라 일 .여기서 난동직이는기 느거일이네....,3,0,1
...,...,...,...,...,...,...
35995,2022-12-28,SK아이이테크놀로지,얜 배당도 안주면서 개폭이네 ㅋ ㅋ,312,1,60
35996,2022-12-28,SK아이이테크놀로지,쌓는거봐라 욕나오네,343,1,60
35997,2022-12-28,SK아이이테크놀로지,"SK 배터리 탑재 아이오닉7,EV9 출시 23년은 SK의해다",478,5,60
35998,2022-12-28,SK아이이테크놀로지,3만원대까지 올까요? 냉철한 분석 부탁삽니다,665,2,60


In [21]:
# 전날 기준 상/하향 열을 추가해주는 코드

def NS_updown():
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
    updown_df = pd.DataFrame([])

    for i in tqdm(range(30)): # 기업1~30위

        list = check_df.loc[i]


        # 각 기업 별 주식 상/하향 사이트에서 가장 마지막 페이지를 추출해준다.
        # page=300으로 설정해놓으면 가장 마지막 페이지의 사이트로 이동한다.
        # 거기서 하단에 있는 가장 마지막 페이지 번호를 가져온다.

        url = "https://finance.naver.com/item/frgn.nhn?code=%s&page=%s" % (list[3],str(300)) # list[3]은 기업명
        html = requests.get(url, headers=headers).content
        # 한글 깨짐 방지 decode
        soup = bs(html.decode('euc-kr', 'replace'), 'html.parser')
        last_page_element = soup.find("td", class_="on")
        last_page_number = int(last_page_element.a.text) # 가장 마지막 페이지
        

        for page in range(1,last_page_number+1): # 각 기업의 시세 상/하락 사이트에서 page를 1 ~ 마지막 페이지까지 탐색

            date_list = [] # 상/하향 적용된 날짜를 담아두는 리스트
            updown_list = [] # 상/하향 여부를 담아두는 리스트     
            
            url = "https://finance.naver.com/item/frgn.nhn?code=%s&page=%s" % (list[3],str(page)) # 해당 기업의 일별 시세 정보방

            html = requests.get(url, headers=headers).content
            # 한글 깨짐 방지 decode
            soup = bs(html.decode('euc-kr', 'replace'), 'html.parser')
            table = soup.select_one("#content > div.section.inner_sub > table.type2")

            span_tags = table.select("span.tah.p10.gray03") # 해당 페이지의 상/하락 날짜의 개수
            
            # 가끔 가다가 상/하락이 아니라 그대로 유지인 경우가 있다.
            # 상/하락은 img정보에 상/하락 여부가 있어 img의 text를 추출하먄 상향/하향 구분되어 나오지만
            # 유지되는 경우는 lmg가 없어서 그대로 스킵되고 하나씩 밀리기 시작한다......
            # 그 때 스킵되는게 아니라 0(nan)값을 호출해줄 수 있도록 한다.
            rows = table.find_all("tr")
            for row in rows:
                td = row.find("td", {"class": "num"})
                if td and td["width"] == "67":
                    img = row.find("img")
                    if img:
                        updown_list.append(img.get("alt", ""))
                    else:
                        updown_list.append("0")
            
            #결과적으로 updown_list의 원소 개수와 span_tags(상/히락 일자 개수)는 동일해지게 된다.
            # 밑에서 for문으로 돌려 각자 생성된 list[i]를 적용해 호출

            # 상/하향 일자의 개수(num)만큼 반복한다.
            num_of_date = len(span_tags)
            num = num_of_date
            for i in range(num): 
                span_texts = [span.text.strip() for span in span_tags][i] # 주식 상/하락 기록된 날짜
                date_list.append(span_texts) 

                table = pd.DataFrame({'날짜': [date_list[i]], '기업명':[list[2]], '전날_대비_상승_하락' : [updown_list[i]]})
                updown_df = updown_df.append(table)

    updown_df["날짜"] = pd.to_datetime(updown_df['날짜'], format = "%Y.%m.%d")
    return updown_df


In [22]:
# 날짜 / 기업명 / 상/하락 여부로 이루어진 dataframe을 생성한다. (이는 나중에 target_data로 활용)
# 또한 이 datefarme을 밑에서 원본 dataframe과 merge하여 원본 dataframe에 '전날 기준 상/하락 여부' 열을 추가한다.

data_updown = NS_updown() # 20분 정도 걸린다.

mapping = {'상승' : 1, '하락' : -1}

data_updown['전날_대비_상승_하락'] = data_updown['전날_대비_상승_하락'].map(mapping)

data_updown = data_updown.reset_index(drop=True)

data_updown['전날_대비_상승_하락'] = data_updown['전날_대비_상승_하락'].fillna(0)

data_updown['전날_대비_상승_하락'] = data_updown['전날_대비_상승_하락'].astype(int)

data_updown

100%|██████████| 30/30 [18:40<00:00, 37.35s/it]


,날짜,기업명,전날_대비_상승_하락
0,2023-02-09,삼성전자,-1
1,2023-02-08,삼성전자,1
2,2023-02-07,삼성전자,1
3,2023-02-06,삼성전자,-1
4,2023-02-03,삼성전자,1
...,...,...,...
94887,2021-05-17,SK아이이테크놀로지,-1
94888,2021-05-14,SK아이이테크놀로지,-1
94889,2021-05-13,SK아이이테크놀로지,-1
94890,2021-05-12,SK아이이테크놀로지,-1


In [23]:
data_updown['전날_대비_상승_하락'].unique()

array([-1,  1,  0])

In [24]:
# target_1.csv는 네이버 일일시세의 기업 별로 전체 모든 기간에 대한 일자 별 상/하락 여부를 긁어모은 것
# 네이버 일일시세에는 주말이나 공휴일은 포함되어있지 않아 해당 일은 skip되어 있다.

data_updown.to_csv('target_1.csv') # target_data를 저장

In [25]:
data_updown = pd.read_csv('target_1.csv',lineterminator='\n').drop(['Unnamed: 0'], axis=1)

In [26]:
data_updown

,날짜,기업명,전날_대비_상승_하락
0,2023-02-09,삼성전자,-1
1,2023-02-08,삼성전자,1
2,2023-02-07,삼성전자,1
3,2023-02-06,삼성전자,-1
4,2023-02-03,삼성전자,1
...,...,...,...
94887,2021-05-17,SK아이이테크놀로지,-1
94888,2021-05-14,SK아이이테크놀로지,-1
94889,2021-05-13,SK아이이테크놀로지,-1
94890,2021-05-12,SK아이이테크놀로지,-1


In [27]:
# 원본 데이터와 target data를 merge해 원본 데이터에 '전날 기준 상/하락 여부' 열을 추가 위해 복사본 생성

data_total = data.copy()

In [28]:
data_total

,날짜,기업명,제목+내용,조회수,공감수,페이지
0,2023-02-09,삼성전자,50억은 무죄야 그거 재개발 이익에서 몇푼 안되거든.\r 나눠 먹은 애들이 아직...,1,0,1
1,2023-02-09,삼성전자,누가 한민족 아니랄까봐 ㅋㅋㅋㅋㅋ 북한은 돼지딸 때매 난리고\r 남한은 개딸 때매...,5,0,1
2,2023-02-09,삼성전자,노인들 무임승차 이슈는 그냥 분노를 노인들한테 돌리는거야 뻔하지 뭐\r 야비한 수법,4,0,1
3,2023-02-09,삼성전자,2찍이들 몰살 해야해~~~~~~~~ 그래야 나라가 정상으로 간당,4,1,1
4,2023-02-09,삼성전자,빨개이들 이리와 오전 빠따마짜!!!\r일해라 일 .여기서 난동직이는기 느거일이네....,3,0,1
...,...,...,...,...,...,...
35995,2022-12-28,SK아이이테크놀로지,얜 배당도 안주면서 개폭이네 ㅋ ㅋ,312,1,60
35996,2022-12-28,SK아이이테크놀로지,쌓는거봐라 욕나오네,343,1,60
35997,2022-12-28,SK아이이테크놀로지,"SK 배터리 탑재 아이오닉7,EV9 출시 23년은 SK의해다",478,5,60
35998,2022-12-28,SK아이이테크놀로지,3만원대까지 올까요? 냉철한 분석 부탁삽니다,665,2,60


In [29]:
# merge할 때 datetime 있으면 안됨.
data_total["날짜"] = data_total["날짜"].str.replace("-", "").astype(int)

In [30]:
data_updown["날짜"] = data_updown["날짜"].str.replace("-", "").astype(int)

In [38]:
# 주식 상/하락 기록된 data_updown과 기존 데이터를 복사했던 data_total을 merge한다.

 # date_total에는 없고, data_updown에는 있는 날짜들도 전부 추가시키고 그 날짜들의 다른 열의 값들은 nan값이 채워진다.
data_merge = pd.merge(data_total, data_updown, on = ['날짜','기업명'], how = 'outer')

# 상/하락 열에서 생성된 빈 날짜 별 값들은 0으로 채워준다.
data_merge['전날_대비_상승_하락'] = data_merge['전날_대비_상승_하락'].fillna(0)

# nan값이 포함된 행을 전부 삭제한다. 이 과정에서 data_total에 새로 추가된 날짜들이 삭제된다.
data_merge = data_merge.dropna(axis=0)

# 값이 float화 되서 int로 바꾼다.
data_merge['전날_대비_상승_하락'] = data_merge['전날_대비_상승_하락'].astype(int)
data_merge['조회수'] = data_merge['조회수'].astype(int)
data_merge['공감수'] = data_merge['공감수'].astype(int)
data_merge['페이지'] = data_merge['페이지'].astype(int)

# 다시 날짜를 datetime 형식으로 변경
data_merge['날짜'] = data_merge['날짜'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

# 제목 + 내용에서 '\r' 제거
data_merge["제목+내용"] = data_merge["제목+내용"].str.replace("\r", "")

In [39]:
data_merge.isnull().sum()

날짜             0
기업명            0
제목+내용          0
조회수            0
공감수            0
페이지            0
전날_대비_상승_하락    0
dtype: int64

In [40]:
data_merge['전날_대비_상승_하락'].unique()

array([-1,  1,  0])

In [41]:
data_merge

,날짜,기업명,제목+내용,조회수,공감수,페이지,전날_대비_상승_하락
0,2023-02-09,삼성전자,50억은 무죄야 그거 재개발 이익에서 몇푼 안되거든. 나눠 먹은 애들이 아직 안...,1,0,1,-1
1,2023-02-09,삼성전자,누가 한민족 아니랄까봐 ㅋㅋㅋㅋㅋ 북한은 돼지딸 때매 난리고 남한은 개딸 때매 난리네,5,0,1,-1
2,2023-02-09,삼성전자,노인들 무임승차 이슈는 그냥 분노를 노인들한테 돌리는거야 뻔하지 뭐 야비한 수법,4,0,1,-1
3,2023-02-09,삼성전자,2찍이들 몰살 해야해~~~~~~~~ 그래야 나라가 정상으로 간당,4,1,1,-1
4,2023-02-09,삼성전자,빨개이들 이리와 오전 빠따마짜!!!일해라 일 .여기서 난동직이는기 느거일이네.더 ...,3,0,1,-1
...,...,...,...,...,...,...,...
35995,2022-12-28,SK아이이테크놀로지,얜 배당도 안주면서 개폭이네 ㅋ ㅋ,312,1,60,-1
35996,2022-12-28,SK아이이테크놀로지,쌓는거봐라 욕나오네,343,1,60,-1
35997,2022-12-28,SK아이이테크놀로지,"SK 배터리 탑재 아이오닉7,EV9 출시 23년은 SK의해다",478,5,60,-1
35998,2022-12-28,SK아이이테크놀로지,3만원대까지 올까요? 냉철한 분석 부탁삽니다,665,2,60,-1


In [42]:
# '\r'제거도 추가됨 원본 + 전날기준상하락

data_merge.to_csv('네이버_60페이지_상승하락적용.csv')

In [43]:
data_merge = pd.read_csv('네이버_60페이지_상승하락적용.csv',lineterminator='\n').drop(['Unnamed: 0'], axis=1)

In [44]:
data_merge

,날짜,기업명,제목+내용,조회수,공감수,페이지,전날_대비_상승_하락
0,2023-02-09,삼성전자,50억은 무죄야 그거 재개발 이익에서 몇푼 안되거든. 나눠 먹은 애들이 아직 안...,1,0,1,-1
1,2023-02-09,삼성전자,누가 한민족 아니랄까봐 ㅋㅋㅋㅋㅋ 북한은 돼지딸 때매 난리고 남한은 개딸 때매 난리네,5,0,1,-1
2,2023-02-09,삼성전자,노인들 무임승차 이슈는 그냥 분노를 노인들한테 돌리는거야 뻔하지 뭐 야비한 수법,4,0,1,-1
3,2023-02-09,삼성전자,2찍이들 몰살 해야해~~~~~~~~ 그래야 나라가 정상으로 간당,4,1,1,-1
4,2023-02-09,삼성전자,빨개이들 이리와 오전 빠따마짜!!!일해라 일 .여기서 난동직이는기 느거일이네.더 ...,3,0,1,-1
...,...,...,...,...,...,...,...
35995,2022-12-28,SK아이이테크놀로지,얜 배당도 안주면서 개폭이네 ㅋ ㅋ,312,1,60,-1
35996,2022-12-28,SK아이이테크놀로지,쌓는거봐라 욕나오네,343,1,60,-1
35997,2022-12-28,SK아이이테크놀로지,"SK 배터리 탑재 아이오닉7,EV9 출시 23년은 SK의해다",478,5,60,-1
35998,2022-12-28,SK아이이테크놀로지,3만원대까지 올까요? 냉철한 분석 부탁삽니다,665,2,60,-1


In [56]:
target_2 = pd.DataFrame([])
for company in tqdm(data_merge['기업명'].unique()):
    for date in data_merge[data_merge['기업명']==company]['날짜'].unique():
        target = pd.DataFrame({'날짜': [date], '기업명':[company], '전날_대비_상승_하락' : data_merge[(data_merge['날짜']==date) & (data_merge['기업명']==company)]['전날_대비_상승_하락'].unique()[0]})
        target_2 = target_2.append(target)

target_2 = target_2.reset_index(drop=True)
target_2

100%|██████████| 30/30 [00:11<00:00,  2.55it/s]


,날짜,기업명,전날_대비_상승_하락
0,2023-02-09,삼성전자,-1
1,2023-02-08,삼성전자,1
2,2023-02-09,카카오,1
3,2023-02-08,카카오,1
4,2023-02-07,카카오,1
...,...,...,...
2867,2023-01-01,SK아이이테크놀로지,0
2868,2022-12-31,SK아이이테크놀로지,0
2869,2022-12-30,SK아이이테크놀로지,0
2870,2022-12-29,SK아이이테크놀로지,-1


In [61]:
# target_2.csv는 원본 데이터를 기준으로 해당 데이터에 존재하는 일자 별 기업 별 '전날대비상하락'이 표시된다.
# 주말이나 공휴일이 포함될 경우 상하락 여부는 0으로 표시됨.
# 기업 별 60페이지 이내에 있는 기간만 해당된다. (ex. 삼성전자는 글이 너무 많아서 60페이지 안에 해당되는 날짜가 2023-02-08, 2023-02-09가 전부)
# 특정 기업의 어느 날 글이 안올라왔으면 그 날의 데이터는 존재 x
target_2.to_csv('target_2.csv')

In [62]:
target_2 = pd.read_csv('target_2.csv',lineterminator='\n').drop(['Unnamed: 0'], axis=1)

target_2

,날짜,기업명,전날_대비_상승_하락
0,2023-02-09,삼성전자,-1
1,2023-02-08,삼성전자,1
2,2023-02-09,카카오,1
3,2023-02-08,카카오,1
4,2023-02-07,카카오,1
...,...,...,...
2867,2023-01-01,SK아이이테크놀로지,0
2868,2022-12-31,SK아이이테크놀로지,0
2869,2022-12-30,SK아이이테크놀로지,0
2870,2022-12-29,SK아이이테크놀로지,-1
